In [1]:
import openai
#import config  # Import your config.py file this contains you openai api key
import pandas as pd
import numpy as np
import os
import time
from datetime import datetime, timedelta
import tiktoken
import re
import difflib
# Set up the OpenAI API key from the .env file, this allows you to keep your key secret and not expose on github
#have the api key in like the below
#OPENAI_API_KEY = "my api key"

import time
import anthropic
from llm_caller import RateLimiter, get_response_openai, get_response_anthropic, make_api_call_from_dataframe_row


In [3]:
OCR_text = f"""fcm E ¥ gNlGP «® WOTH ) THE R&PP ^ Ol SPBRBTTS . [ The following Letter was written at our request . The writer , a German friend , whose integrity and clear-headedness would command attention to whatever he might assert , informed us of his proposed visit to a house where the Rappitcs were to exhibit , and we begged him to furnish us with a report . We print his letter without any alteration : its statements will , we think , set the reader speculating .Let it be distinctly borne in mind , however , that in printing this Letter we give no countenance whatever to the pretended explanation of " Rapping . " We have our own views of the jugglery by which the "ghosts " are made to communicate with persons willing to pay for the interview , and in a very short time we hope to lay before our readers something like an experimental result . ] My dear Sir , —When I proposed to give you a report on these new prophets , who , if as true as they arc new , open to us a wide and most interesting field for inquiry , overthrow ancient and modem systems of science and belief , shake to the very foundation revealed religion andChristianity , hut , on the other hand , arc telling almost equally strong against Pantheism mid Atheism , I had not seen those prophets , expounders , mediums , or whatever you may call them , myself ; hut what I heard from a friend , a clear-sighted , well-informed , by no moans " gullible , " or over-credulous gentleman , who had paid them ii visit , had made me anxious to see and judge for myself ; and he having determined , for better satisfaction , to have the Medium ( and the spirits ) at his own house , and having kindly invited me to be present on the occasion , 1 offered to furnish you with a statement of the result of the evening , which I now , agreeably to your wish , lay before you .We were five of us in the library , —my friend , his wife , his sister , his nephew , and myself , —when the footman , handing in a card , announced "Mrs . JIayden . " Her entrance and deportment were easy , unembarrassed , and yet not " business-like ; " her exterior rather prepossessing ; an intelligent countenance , with , perhaps , a slight touch of Yankccisin in the corner of the eye ; and the conversation soon bring established , showed that she did not lack those powers of speech so peculiar to the . citizens of the great Republic ; though certain mistakes now and then recurring in the conjugation of verbs , indicated that nhe could not have had a very fin ;! -rate education . We took our seats round the table on which the card had been placed , [ mid , — "Mr . and Mrs . W . R . Jlirydeii , L' 2 , Queen Anne-street , Cavenuish-. squarc . " Expectation created silence , now and then broken by . qiiestions in a low voice , addressed to the Medium , and by the wheels of carriages that brought guests to an evening party at a neighbouring house , and thus tnadd the otherwise very quirt sired ; liither noisy . Tho raps which the Medium and one or two of us , after we had been seated for about ten"""

In [4]:
system_message = f"""Please recover the text from the corrupted OCR, providing appropriate paragraph breaks. 
Do not add any text, commentary, or lead in sentences  beyond the recovered text. Do not add a title, or any introductions"""

prompt = OCR_text + "\n\n" + f"""Please recover the text from the corrupted OCR, providing appropriate paragraph breaks. Do not add any text, commentary, or lead in sentences  beyond the recovered text"""

system_message = ""

#prompt = OCR_text

In [25]:
client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=os.getenv("ANTHR_API_KEY"),
)

message = client.messages.create(
    model="claude-3-haiku-20240307",#"claude-3-opus-20240229",
    max_tokens=4000,
    temperature=0.0,
    system=system_message,
    messages=[
        {"role": "user", "content": prompt}
    ]
)

print(message.content)

[ContentBlock(text='Here is the text recovered from the corrupted OCR, with appropriate paragraph breaks:\n\nMy dear Sir, —When I proposed to give you a report on these new prophets, who, if as true as they are new, open to us a wide and most interesting field for inquiry, overthrow ancient and modern systems of science and belief, shake to the very foundation revealed religion and Christianity, but, on the other hand, are telling almost equally strong against Pantheism and Atheism, I had not seen those prophets, expounders, mediums, or whatever you may call them, myself; but what I heard from a friend, a clear-sighted, well-informed, by no means "gullible," or over-credulous gentleman, who had paid them a visit, had made me anxious to see and judge for myself; and he having determined, for better satisfaction, to have the Medium (and the spirits) at his own house, and having kindly invited me to be present on the occasion, I offered to furnish you with a statement of the result of the

In [47]:
message.usage.input_tokens + message.usage.output_tokens

1313

In [6]:
rate_limiter = RateLimiter(50000)

test = get_response_anthropic(prompt, system_message, rate_limiter, engine="claude-3-haiku-20240307", max_tokens = 4000)

In [7]:
test

'Here is the text recovered from the corrupted OCR, with appropriate paragraph breaks:\n\nMy dear Sir, —When I proposed to give you a report on these new prophets, who, if as true as they are new, open to us a wide and most interesting field for inquiry, overthrow ancient and modern systems of science and belief, shake to the very foundation revealed religion and Christianity, but, on the other hand, are telling almost equally strong against Pantheism and Atheism, I had not seen those prophets, expounders, mediums, or whatever you may call them, myself; but what I heard from a friend, a clear-sighted, well-informed, by no means "gullible," or over-credulous gentleman, who had paid them a visit, had made me anxious to see and judge for myself; and he having determined, for better satisfaction, to have the Medium (and the spirits) at his own house, and having kindly invited me to be present on the occasion, I offered to furnish you with a statement of the result of the evening, which I n

In [26]:
message

Message(id='msg_01RDuf2hzxmMSCQHRQkQT9XH', content=[ContentBlock(text='Here is the text recovered from the corrupted OCR, with appropriate paragraph breaks:\n\nMy dear Sir, —When I proposed to give you a report on these new prophets, who, if as true as they are new, open to us a wide and most interesting field for inquiry, overthrow ancient and modern systems of science and belief, shake to the very foundation revealed religion and Christianity, but, on the other hand, are telling almost equally strong against Pantheism and Atheism, I had not seen those prophets, expounders, mediums, or whatever you may call them, myself; but what I heard from a friend, a clear-sighted, well-informed, by no means "gullible," or over-credulous gentleman, who had paid them a visit, had made me anxious to see and judge for myself; and he having determined, for better satisfaction, to have the Medium (and the spirits) at his own house, and having kindly invited me to be present on the occasion, I offered t

fcm E ¥ gNlGP «® WOTH ) THE R&PP ^ Ol SPBRBTTS . [ The following Letter was written at our request . The writer , a German friend , whose integrity and clear-headedness would command attention to whatever he might assert , informed us of his proposed visit to a house where the Rappitcs were to exhibit , and we begged him to furnish us with a report . We print his letter without any alteration : its statements will , we think , set the reader speculating .Let it be distinctly borne in mind , however , that in printing this Letter we give no countenance whatever to the pretended explanation of " Rapping . " We have our own views of the jugglery by which the "ghosts " are made to communicate with persons willing to pay for the interview , and in a very short time we hope to lay before our readers something like an experimental result . ] My dear Sir , —When I proposed to give you a report on these new prophets , who , if as true as they arc new , open to us a wide and most interesting field for inquiry , overthrow ancient and modem systems of science and belief , shake to the very foundation revealed religion andChristianity , hut , on the other hand , arc telling almost equally strong against Pantheism mid Atheism , I had not seen those prophets , expounders , mediums , or whatever you may call them , myself ; hut what I heard from a friend , a clear-sighted , well-informed , by no moans " gullible , " or over-credulous gentleman , who had paid them ii visit , had made me anxious to see and judge for myself ; and he having determined , for better satisfaction , to have the Medium ( and the spirits ) at his own house , and having kindly invited me to be present on the occasion , 1 offered to furnish you with a statement of the result of the evening , which I now , agreeably to your wish , lay before you .We were five of us in the library , —my friend , his wife , his sister , his nephew , and myself , —when the footman , handing in a card , announced "Mrs . JIayden . " Her entrance and deportment were easy , unembarrassed , and yet not " business-like ; " her exterior rather prepossessing ; an intelligent countenance , with , perhaps , a slight touch of Yankccisin in the corner of the eye ; and the conversation soon bring established , showed that she did not lack those powers of speech so peculiar to the . citizens of the great Republic ; though certain mistakes now and then recurring in the conjugation of verbs , indicated that nhe could not have had a very fin ;! -rate education . We took our seats round the table on which the card had been placed , [ mid , — "Mr . and Mrs . W . R . Jlirydeii , L' 2 , Queen Anne-street , Cavenuish-. squarc . " Expectation created silence , now and then broken by . qiiestions in a low voice , addressed to the Medium , and by the wheels of carriages that brought guests to an evening party at a neighbouring house , and thus tnadd the otherwise very quirt sired ; liither noisy . Tho raps which the Medium and one or two of us , after we had been seated for about ten

Please recover the text from the corrupted OCR, providing appropriate paragraph breaks. Do not add any text, commentary, or lead in sentences  beyond the recovered text, for example for the text

"Md BRO5n0n D1e] t@ ay"

The correct response would be

"Mr Bronson Died today"

An incorrect response would be 

"Sure here is the recovered text: Mr Bronson died today"

Another incorrect response would be

"Here is the recovered text from the corrupted OCR: Mr Bronson died today"

In both cases a lead in sentence was generated which is why the example responses were incorrect

